# Analysis of STAP-seq data

In [1]:
import requests
import pandas as pd
import numpy as np

In [3]:
# Download zfh1 count, ctrl count and enhancer-responsiveness
url="https://data.starklab.org/publications/arnold_zabidi_natureBiotech_2016/plus_1_pos.txt"
r = requests.get(url, allow_redirects=True)

file = 'positions/plus_1_pos.txt'

open(file,'wb').write(r.content)

eTSSrpve = pd.read_csv(file, sep="\t", header=None)[1:]
eTSSrpve.columns = ["chr", "start", "end", "TSS_num","strand","zfh1_count","ctrl_count","responsiveness"]
eTSSrpve.head()

,chr,start,end,TSS_num,strand,zfh1_count,ctrl_count,responsiveness
1,chr2L,12815,12816,TSS_10945,+,5,0,1.75509432621
2,chr2L,59243,59244,TSS_5268,-,14,0,3.71122827673
3,chr2L,67043,67044,TSS_2980,+,39,1,9.17713509827
4,chr2L,72599,72600,TSS_6539,+,10,0,2.83881166131
5,chr2L,84184,84185,TSS_11273,-,5,0,1.75509432621


In [7]:
#download subthreshold position

colnames = ["chr", "start", "end", "TSS_num","zfh1_count","strand","responsiveness"]

for sub in [2, 3, 4]:    
    
    url=f"https://data.starklab.org/publications/arnold_zabidi_natureBiotech_2016/positions_zfh1_{sub}.txt"
    r = requests.get(url, allow_redirects=True)
    
    file = f"positions/positions_zfh1_{sub}.txt"
    open(file,'wb').write(r.content)
    subpos = pd.read_csv(file, sep="\t", header=None)
    subpos.columns = colnames
    
# Look at the subthreshold 4 positions
subpos.head()

,chr,start,end,TSS_num,zfh1_count,strand,responsiveness
0,chr2L,444142,444243,position_1757,4,-,1.541716
1,chr2L,536556,536657,position_2360,4,+,1.541716
2,chr2L,775947,776048,position_3457,4,+,1.541716
3,chr2L,779122,779223,position_3466,4,+,1.541716
4,chr2L,1153617,1153718,position_5084,4,+,1.541716


In [8]:
#download random positions
url="https://data.starklab.org/publications/arnold_zabidi_natureBiotech_2016/positions_random.txt"
r = requests.get(url, allow_redirects=True)

file = "positions/positions_random.txt"
open(file,'wb').write(r.content)
rand_pos = pd.read_csv(file, sep="\t", header=None)

rand_pos.columns = ["chr", "start", "end", "TSS_num","null","strand","responsiveness"]
rand_pos.head()

,chr,start,end,TSS_num,null,strand,responsiveness
0,chr2L,39268,39369,rTSS_1,1,-,1.0
1,chr2L,54484,54585,rTSS_3,1,+,1.0
2,chr2L,54812,54913,rTSS_4,1,-,1.0
3,chr2L,68702,68803,rTSS_5,1,+,1.0
4,chr2L,70511,70612,rTSS_6,1,+,1.0


In [ ]:
#download the sequence around the TSSes
!grabSequences_220920.sh

In [9]:
df2 = pd.read_csv('sequences/positons_zfh1_2_seq.txt.gz', compression='gzip', header=None,sep='\t')
df3 = pd.read_csv('sequences/positons_zfh1_3_seq.txt.gz', compression='gzip', header=None,sep='\t')
df4 = pd.read_csv('sequences/positons_zfh1_4_seq.txt.gz', compression='gzip', header=None,sep='\t')
df_eTSS = pd.read_csv('sequences/plus_1_pos_seq.txt.gz', compression='gzip',sep='\t')
df2.head()

,0,1,2,3,4,5
0,chr2L,12678,12878,position_13,2,ACCGGCGTTCAGCTGATCGACAGGGCTACCCGCCTGATCAAAAATA...
1,chr2L,14570,14770,position_24,2,GTACGACTTCAGAAGCAAGGTAATTGCAAGTGACAGCTGATGCATG...
2,chr2L,14677,14877,position_25,2,CAGAGTTCCTCTTCAAGTAGTACAACAAGAACTTCAGCGACATCTC...
3,chr2L,14716,14916,position_26,2,CACAAAGTTTTTTAGATGTGTACTAATTTCTCGTTCTTTTAGATCC...
4,chr2L,21682,21882,position_31,2,CTTTAGTGCACATGAAATTATATGTATTGAAATAAAAAGTTTGATG...


In [10]:
df_eTSS.columns=['chr','start','end','TSS_num','responsiveness','strand','sequence']
df_eTSS.head()

,chr,start,end,TSS_num,responsiveness,strand,sequence
0,chr2L,59194,59294,TSS_5268,3.711228,-,AACTTGGTTGGCTGGCTGCGCTTGCCCAGAGTCTCAAGACACTCGT...
1,chr2L,66994,67094,TSS_2980,9.177135,+,TGTGTGAGTGGTGAGTGTAATGGCAAACGGCAGCAACATTTATTTC...
2,chr2L,72550,72650,TSS_6539,2.838812,+,AAAACCGCATTTGCATCGGGTAAAATCGGTAACCCCATCTTTGCAT...
3,chr2L,84135,84235,TSS_11273,1.755094,-,CCAGTTCGGAAAGCGAAGGCGGAGACGACTTGGTCGTCGAGCACGC...
4,chr2L,87332,87432,TSS_2518,12.021107,-,GAAGCCAGCCGGTCCCTGGCGCAGGCCATTTCAATCCGCAAGATAT...


In [81]:
#convert into 5mers
#chop the sequence into 5-mers
mers=5
def chopseq(line):return [line[i:i+mers] for i in range(0, len(line), mers)]

df_eTSS['five_mers_list']=df_eTSS['sequence'].map(chopseq)

#convert list of 5-mers into a long string with spaces in between
def listToString(s):  
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 

# df_eTSS['five_mers_str']=df_eTSS['five_mers_list'].map(listToString)

#remove nucleotide shorter than 5-mers
def remove_short(line):
    new_string = ' '.join([w for w in line.split() if len(w)==mers])
    return new_string

# df_eTSS['five_mers_str']=df_eTSS['five_mers_str'].map(remove_short)

df_eTSS.head(1)

,chr,start,end,TSS_num,responsiveness,strand,sequence,five_mers_list
0,chr2L,59194,59294,TSS_5268,3.711228,-,AACTTGGTTGGCTGGCTGCGCTTGCCCAGAGTCTCAAGACACTCGT...,"[AACTT, GGTTG, GCTGG, CTGCG, CTTGC, CCAGA, GTC..."
